<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/Creating_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# הורדת הקובץ מ-GitHub
!wget -O prepare_data.py "https://raw.githubusercontent.com/orifelszer/CrimeData/main/prepare_data.py"

# ייבוא הפונקציה מהקובץ
from prepare_data import Preprocessing

--2025-01-07 16:20:06--  https://raw.githubusercontent.com/orifelszer/CrimeData/main/prepare_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721 (4.6K) [text/plain]
Saving to: ‘prepare_data.py’

prepare_data.py     100%[===================>]   4.61K  --.-KB/s    in 0s      

2025-01-07 16:20:07 (39.9 MB/s) - ‘prepare_data.py’ saved [4721/4721]



Define Paths to Crime Datasets (2019–2024)

In [3]:
import os
import zipfile

# הורדת הנתונים מה-GitHub
!git clone https://github.com/orifelszer/CrimeData.git

zip_folder = 'CrimeData'
# ✅ קריאה רק לקבצי ZIP לפי השנים 2019 עד 2024
zip_files = [f for f in os.listdir(zip_folder) if f.startswith('crimes') and any(str(year) in f for year in range(2019, 2025))]

# ✅ פריסת הקבצים ושמירה של נתיבי התיקיות החדשים
extracted_folders = []

for zip_file in zip_files:
    zip_path = os.path.join(zip_folder, zip_file)
    extract_path = os.path.join(zip_folder, zip_file.replace('.zip', ''))
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    extracted_folders.append(extract_path)
    print(f"Extracted: {zip_file} -> {extract_path}")


Cloning into 'CrimeData'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 346 (delta 89), reused 18 (delta 18), pack-reused 188 (from 2)
Receiving objects: 100% (346/346), 211.19 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (172/172), done.
Extracted: crimes2023.zip -> CrimeData/crimes2023
Extracted: crimes2020.zip -> CrimeData/crimes2020
Extracted: crimes2021.zip -> CrimeData/crimes2021
Extracted: crimes2024.zip -> CrimeData/crimes2024
Extracted: crimes2019.zip -> CrimeData/crimes2019
Extracted: crimes2022.zip -> CrimeData/crimes2022


Load Crime Data into Pandas DataFrames

In [4]:
# ✅ איחוד קובצי CSV רק מהתיקיות שחולצו
csv_files = []
for folder in extracted_folders:
    csv_files += glob.glob(os.path.join(folder, "*.csv"))  # חיפוש כל קובצי ה-CSV בתיקיות שחולצו בלבד

In [5]:
# ✅ קריאת כל קובצי ה-CSV ושמירתם במילון לפי שם הקובץ
dataframes = {f"Crimes_{file[-8:-4]}": pd.read_csv(file) for file in csv_files}

In [6]:
# ✅ איחוד כל קובצי ה-CSV לקובץ אחד
combined_data = pd.concat(dataframes.values(), axis=0, ignore_index=True)

In [7]:
combined_data

,FictiveIDNumber,Year,Quarter,YeshuvKod,Yeshuv,PoliceDistrictKod,PoliceDistrict,PoliceMerhavKod,PoliceMerhav,PoliceStationKod,PoliceStation,municipalKod,municipalName,StatisticAreaKod,StatisticArea,StatisticGroupKod,StatisticGroup,StatisticTypeKod,StatisticType
0,5600DF6839DB0293D9154C6E1AC27AA9,2023,Q3,4000.0,חיפה,80000000,מחוז חוף,15005216,מרחב כרמל חוף,81100000,תחנת חיפה מחוז חוף,NaN,NaN,40000942.0,רמת אלמוגי,700,עבירות כלפי הרכוש,712,גניבה מתוך רכב
1,E644D2A75070B162F98C00F319ABA8A8,2023,Q2,8300.0,ראשון לציון,40000000,מחוז מרכז,41200000,מרחב שפלה,41210000,תחנת ראשון לציון שפלה,NaN,NaN,83000311.0,נחלת יהודה,200,עבירות סדר ציבורי,206,איומים
2,1B4ECD6B29AB86E65B1F3A9BC520F382,2023,Q3,NaN,NaN,70000000,מחוז שי,71400000,מרחב יהודה שי,71400800,תחנת עציון יהודה,NaN,ישוב פלסטיני,NaN,NaN,200,עבירות סדר ציבורי,200,עבירות סדר ציבורי
3,E4E881C7378790653D0D3D6CFF1E951D,2023,Q4,NaN,NaN,70000000,מחוז שי,71500000,מרחב שומרון שי,15004488,תחנת אריאל מרחב שומרון,NaN,ישוב פלסטיני,NaN,NaN,200,עבירות סדר ציבורי,200,עבירות סדר ציבורי
4,6D52AF3AF587052B0A230986D80688E5,2023,Q3,7000.0,לוד,40000000,מחוז מרכז,41200000,מרחב שפלה,41213000,תחנת לוד שפלה,NaN,NaN,70000013.0,העיר העתיקה (מערב),100,עבירות בטחון,106,עבירות אחרות כלפי בטחון
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341168,BA52AC818358FA19FD7A9CB5DB60A42C,2022,Q2,3769.0,הר אדר,50000000,מחוז ירושלים,51200000,מרחב ציון ירושלים,51212000,תחנת הראל ציון,NaN,NaN,NaN,NaN,200,עבירות סדר ציבורי,206,איומים
2341169,059690968F6C2F1B81C5C53CCCEFBB5E,2022,Q3,3000.0,ירושלים,50000000,מחוז ירושלים,51200000,מרחב ציון ירושלים,51214000,תחנת לב הבירה ציון,NaN,NaN,30000832.0,מקור ברוך (מערב),700,עבירות כלפי הרכוש,715,גניבות אחרות
2341170,5E4A19D0471BA7E8FC1CD14B473DAD4F,2022,Q3,812.0,שלומי,80000000,מחוז חוף,15002761,מרחב אשר חוף,81501000,תחנת נהריה מחוז חוף,NaN,NaN,NaN,NaN,700,עבירות כלפי הרכוש,715,גניבות אחרות
2341171,304F64CF83DA884724D9217B2BE3800C,2022,Q3,195.0,קדימה צורן,40000000,מחוז מרכז,41100000,מרחב שרון,41118000,תחנת שדות שרון,NaN,NaN,1950001.0,NaN,700,עבירות כלפי הרכוש,719,גרימת נזק לרכוש בזדון


In [8]:
# הסרת שורות עם ערך -1 בעמודת StatisticGroupKod
combined_data = combined_data[combined_data['StatisticGroupKod'] != -1]

In [9]:
# הסרת עמודות החיזוי מהנתונים
features = combined_data.drop(['StatisticGroup', 'StatisticGroupKod'], axis=1)
target = combined_data['StatisticGroup']

# ✅ חלוקה לסטים: המאפיינים מופרדים מעמודות היעד
X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=42,
    stratify=target
    )

In [10]:
# ✅ שימוש בפונקציה המעודכנת ללמידה רב-משימתית
X_train_cleaned, train_mappings, scaler, label_encoders = Preprocessing(X_train, fit_scaler=True)
X_test_cleaned, _, _, _ = Preprocessing(X_test, train_mappings=train_mappings, scaler=scaler)

In [11]:
# הגדרת עמודות לשמירה על בסיס סט האימון בלבד
columns_to_keep = [col for col in X_train_cleaned.columns if X_train_cleaned[col].sum() >= 10]

# שמירה על עמודות אימון רלוונטיות בלבד
X_train_cleaned = X_train_cleaned[columns_to_keep]

# התאמת סט המבחן רק לפי מה שנקבע באימון
for col in columns_to_keep:
    if col not in X_test_cleaned.columns:
        X_test_cleaned[col] = 0

# הבטחה ששני הסטים באותו סדר עמודות
X_test_cleaned = X_test_cleaned[X_train_cleaned.columns]

In [12]:
# Ensure target variables match the indices of the cleaned datasets
y_train = y_train.loc[X_train_cleaned.index]
y_test = y_test.loc[X_test_cleaned.index]

In [13]:
# Reducing memory usage
def optimize_data_types(df):
    for col in df.columns:
        col_type = df[col].dtype

        # אם מדובר במשתנה קטגוריאלי טקסטואלי
        if col_type == 'object':
            df[col] = df[col].astype('category')

        # אם מדובר במשתנה מספרי רציף
        elif col_type == 'float64':
            df[col] = df[col].astype('float32')

        # אם מדובר במשתנה מספרי שלם
        elif col_type == 'int64':
            df[col] = df[col].astype('int32')
    return df

# החלת הפונקציה על סט הנתונים
X_train_cleaned = optimize_data_types(X_train_cleaned)
X_test_cleaned = optimize_data_types(X_test_cleaned)

In [14]:
# Initialize the label encoder for the single target column
label_encoder = LabelEncoder()

# Fit and transform the training data
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform the test data using the same encoder (no refitting)
y_test_encoded = label_encoder.transform(y_test)

In [15]:
# ✅ Save the cleaned datasets and encoded targets as CSV files
X_train_cleaned.to_csv('X_train_supervised.csv', index=False)
X_test_cleaned.to_csv('X_test_supervised.csv', index=False)
pd.DataFrame(y_train_encoded).to_csv('y_train_supervised.csv', index=False, header=['target'])
pd.DataFrame(y_test_encoded).to_csv('y_test_supervised.csv', index=False, header=['target'])